In [ ]:
import re
from disco.scorers import BooleanScorer
from disco.distributions import LMDistribution
from disco.samplers import AccumulationSampler
from disco.tuners import DPGTuner
from disco.tuners.loggers.console import ConsoleLogger
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    AutoModelForCausalLM
)
import sympy as sp

# Basic interface and experiments with CAP

Define Base LLM a(y) and binary constraint b(y)

In [ ]:
token = "" # your own huggingface token
a = LMDistribution("google/gemma-2b", token=token, LLM=True) # base LLM a(y)
b = lambda s, c: bool(re.search(r"\bamazing\b", s.text)) # hard constraint

In [ ]:
distr = AccumulationSampler(distribution=a, total_size=200000) # Total sample size what you want
samples_a, distr_a = distr.sample(sampling_size=500, context="") # If GPU-extensive, than lower sampling size.
print('AR =', sum([b(s, _) for s in samples_a]) / len(samples_a)) # AR_a

In [ ]:
# Use CAP (context-aware prompt) what you want

CAP = "Next sentence should contain 'amazing'.\n\n"

# CAP = "Write sentences with the given words.\n"
# CAP += "diagnosis: Assessment of microscopical and clinical parameters in the diagnosis of diabetes mellitus.\n"
# CAP += "pandas: Column headings differ in spreadsheet that is merged with pandas data\n"
# CAP += "change: How to change the decimal separator in MS Word?\n"
# CAP += "amazing: "
# CAP

In [ ]:
distr = AccumulationSampler(distribution=a, total_size=50000) 
samples_a2, distr_a2 = distr.sample(sampling_size=250, context=CAP) # sample with CAP
print('AR =', sum([b(s, _) for s in samples_a2]) / len(samples_a2))

$$Z = AR_a = \mathbb{E}_{y \sim a} b(y), Z2 = AR_{a2} = \mathbb{E}_{y \sim a2} b(y)$$

In [ ]:
Z = sum([b(s, _) for s in samples_a]) / len(samples_a)
Z2 = sum([b(s, _) for s in samples_a2]) / len(samples_a2)
print('Z and log(Z):', Z, sp.log(Z))
print("Z' and log(Z'):", Z2, sp.log(Z2))

$$KL(g|a) = \mathbb{E}_{y\sim g} \log \frac{g(y)}{a(y)} = \mathbb{E}_{y\sim g} \log \frac{a(y) b(y)}{Z\ a(y)}= \mathbb{E}_{y\sim g} \log \frac{1}{Z} = -\log Z$$

In [ ]:
-sp.log(Z), -sp.log(Z2)

Estimator: $$\mathbb{E}_{y \sim g} \log \frac{a(y)}{a'(y)} \ and \ \mathbb{E}_{y\sim g2} \log \frac{a(y)}{a'(y)}$$

In [ ]:
score = []
for it, item in enumerate(samples_a):
    if b(item, _):
        score.append(a.log_score([samples_a[it]], context="") - a.log_score([samples_a[it]], context=gop))
        
estimator_g = sum(score)/len(score)

score = []
for it, item in enumerate(samples_a2[:10000]):
    if b(item, _):
        score.append(a.log_score([samples_a2[it]], context="") - a.log_score([samples_a2[it]], context=gop))

estimator_g2 = sum(score)/len(score)

estimator_g, estimator_g2

$$KL(g|a’) = \mathbb{E}_{y\sim g} \log \frac{g(y)}{a'(y)} = \mathbb{E}_{y\sim g} \log \frac{a(y) b(y)}{Z\ a'(y)} = \mathbb{E}_{y\sim g} \log \frac{a(y)}{a'(y)} -\log Z$$

$$KL(g|g’) = \mathbb{E}_{y\sim g} \log \frac{g(y)}{g'(y)} = \mathbb{E}_{y\sim g} \log \frac{a(y) b(y)}{Z}\frac{Z'}{a'(y)b(y)} = \mathbb{E}_{y\sim g} \log \frac{a(y)}{a'(y)}-\log \frac{Z}{Z'}$$

In [ ]:
estimator_g-sp.log(Z), estimator_g-sp.log(Z)+sp.log(Z2)

In [ ]:
print("AR upgrade: ", Z2/Z)
print("KL(g|g')", estimator_g-sp.log(Z)+sp.log(Z2))
print("KL(g'|g)", -estimator_g2-sp.log(Z2)+sp.log(Z))
print("AR of a2: ", Z2)
print("KL(g|a):", -sp.log(Z))
print("KL(g'|a'):", -sp.log(Z2))
print("KL(g|a')", estimator_g-sp.log(Z))

# DPG Training

In [ ]:
token = "" # your own huggingface token
proposal = LMDistribution("google/gemma-2b", token=token, LLM=True)
a2 = LMDistribution("google/gemma-2b", token=token, LLM=True, freeze=False)
b = lambda s, c: bool(re.search(r"\bamazing\b", s.text)) # hard constraint

Define our gold distribution $g(y)$ !!!

In [ ]:
scorer = BooleanScorer(b)
g = proposal * scorer

DPG Training with $disco$ libarary

In [ ]:
tuner = DPGTuner(a2, g,
        context="",
        n_gradient_steps=400,
        n_samples_per_step=10000,
        sampling_size=500,
        scoring_size=500,
        divergence_evaluation_interval=10)

ConsoleLogger(tuner)

tuner.tune()

In [ ]:
a2.save(path="models/amazing/dpg-800k")

Evaluate as above protocol

In [ ]:
token = "" # your own huggingface token
a = LMDistribution("google/gemma-2b", token=token, LLM=True)

distr = AccumulationSampler(distribution=a, total_size=500000)
samples_a, distr_a = distr.sample(sampling_size=500, context="")
print('AR =', sum([b(s, _) for s in samples_a]) / len(samples_a))

In [ ]:
a2 = LMDistribution("models/amazing/dpg-800k", token=token, LLM=True)

distr = AccumulationSampler(distribution=a2, total_size=200000)
samples_a2, distr_a2 = distr.sample(sampling_size=500, context="")
print('AR =', sum([b(s, _) for s in samples_a2]) / len(samples_a2))

In [ ]:
Z = sum([b(s, _) for s in samples_a]) / len(samples_a)
Z2 = sum([b(s, _) for s in samples_a2]) / len(samples_a2)
print('Z and log(Z):', Z, sp.log(Z))
print("Z' and log(Z'):", Z2, sp.log(Z2))

In [ ]:
score = []
for it, item in enumerate(samples_a):
    if b(item, _):
        score.append(a.log_score([samples_a[it]], context="") - a2.log_score([samples_a[it]], context=""))
        
estimator_g = sum(score)/len(score)

score = []
for it, item in enumerate(samples_a2[:10000]):
    if b(item, _):
        score.append(a.log_score([samples_a2[it]], context="") - a2.log_score([samples_a2[it]], context=""))

estimator_g2 = sum(score)/len(score)

In [ ]:
print("AR upgrade: ", Z2/Z)
print("KL(g|g')", estimator_g-sp.log(Z)+sp.log(Z2))
print("KL(g'|g)", -estimator_g2-sp.log(Z2)+sp.log(Z))
print("AR of a2: ", Z2)
print("KL(g|a):", -sp.log(Z))
print("KL(g'|a'):", -sp.log(Z2))
print("KL(g|a')", estimator_g-sp.log(Z))
print("KL(g'|a)", -estimator_g2-sp.log(Z2))

# SFT Training

In [ ]:
token = "" # your own huggingface token
model_name = "google/gemma-2b"

In [ ]:
a = LMDistribution(model_name, token=token, LLM=True)
b = lambda s, c: bool(re.search(r"\bamazing\b", s.text)) # hard constraint

sample a lot of $y$ from $a(y)$

In [ ]:
distr = AccumulationSampler(distribution=a, total_size=800000)
samples_a, distr_a = distr.sample(sampling_size=500, context="")

filter $y$ with $b(y)$ to make dataset representing $g$ distribution

In [ ]:
samples_g = []

for it, item in enumerate(samples_a): # y ~ a
    if b(item, _): # if b(y) = 1
        samples_g.append({'text': item[1]}) # return y

print(len(samples_g))

Supervised fine-tuning with a dataset representing $g$ distribution.

In [ ]:
from trl import SFTTrainer
from datasets import load_dataset, Dataset

training_args = TrainingArguments(
    output_dir='models/sft',
    per_device_train_batch_size=64,
    num_train_epochs=1,
    learning_rate=5e-06,
    report_to="none",
)

ds_train = Dataset.from_list(samples_g) 
ds_train = ds_train.shuffle()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
a = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', trust_remote_code=True)
# a.to('cuda')

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=a,
    args=training_args,
    train_dataset=ds_train,
    max_seq_length=30,
    tokenizer=tokenizer,
    dataset_text_field='text',
)

In [ ]:
trainer.train()
trainer.save_model("models/amazing/sft-800k")

Evaluate as above protocol

In [ ]:
token = "" # your own huggingface token
a = LMDistribution("google/gemma-2b", token=token, LLM=True)

distr = AccumulationSampler(distribution=a, total_size=500000)
samples_a, distr_a = distr.sample(sampling_size=500, context="")
print('AR =', sum([b(s, _) for s in samples_a]) / len(samples_a))

In [ ]:
a2 = LMDistribution("models/amazing/dpg-800k", token=token, LLM=True)

distr = AccumulationSampler(distribution=a2, total_size=200000)
samples_a2, distr_a2 = distr.sample(sampling_size=500, context="")
print('AR =', sum([b(s, _) for s in samples_a2]) / len(samples_a2))

In [ ]:
Z = sum([b(s, _) for s in samples_a]) / len(samples_a)
Z2 = sum([b(s, _) for s in samples_a2]) / len(samples_a2)
print('Z and log(Z):', Z, sp.log(Z))
print("Z' and log(Z'):", Z2, sp.log(Z2))

In [ ]:
score = []
for it, item in enumerate(samples_a):
    if b(item, _):
        score.append(a.log_score([samples_a[it]], context="") - a2.log_score([samples_a[it]], context=""))
        
estimator_g = sum(score)/len(score)

score = []
for it, item in enumerate(samples_a2[:10000]):
    if b(item, _):
        score.append(a.log_score([samples_a2[it]], context="") - a2.log_score([samples_a2[it]], context=""))

estimator_g2 = sum(score)/len(score)

In [ ]:
print("AR upgrade: ", Z2/Z)
print("KL(g|g')", estimator_g-sp.log(Z)+sp.log(Z2))
print("KL(g'|g)", -estimator_g2-sp.log(Z2)+sp.log(Z))
print("AR of a2: ", Z2)
print("KL(g|a):", -sp.log(Z))
print("KL(g'|a'):", -sp.log(Z2))
print("KL(g|a')", estimator_g-sp.log(Z))
print("KL(g'|a)", -estimator_g2-sp.log(Z2))

# Warm-start DPG

In [ ]:
token = "" # your own huggingface token
model_name = "google/gemma-2b"
a = LMDistribution(model_name, token=token, LLM=True)
b = lambda s, c: bool(re.search(r"\bamazing\b", s.text)) # hard constraint

Use CAP to make a lot of $y \sim a(\cdot|CAP)$ with small sampling budget

In [ ]:
CAP = "Next sentence should contain 'amazing'.\n\n"
distr = AccumulationSampler(distribution=a, total_size=10000)
samples_a, distr_a = distr.sample(sampling_size=500, context=CAP)

In [ ]:
samples_g = []

for it, item in enumerate(samples_a): # y ~ a
    if b(item, _): # if b(y) = 1
        samples_g.append({'text': item[1]}) # return y

print(len(samples_g))

fine-tune $y$ to make warm-start model.
Note: Do not make too many gradient signal!! It is biased dataset!!

In [ ]:
from trl import SFTTrainer
from datasets import load_dataset, Dataset

training_args = TrainingArguments(
    output_dir='models/sft',
    per_device_train_batch_size=24,
    num_train_epochs=1,
    learning_rate=5e-06,
    report_to="none",
)

ds_train = Dataset.from_list(samples_g) 
ds_train = ds_train.shuffle()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
a = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', trust_remote_code=True)
# a.to('cuda')

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=a,
    args=training_args,
    train_dataset=ds_train,
    max_seq_length=30,
    tokenizer=tokenizer,
    dataset_text_field='text',
)

In [ ]:
trainer.train()
trainer.save_model("models/amazing/ws")

In [ ]:
token = "" # your own huggingface token
proposal = LMDistribution("google/gemma-2b", token=token, LLM=True)
a2 = LMDistribution("models/amazing/ws", token=token, LLM=True, freeze=False)
b = lambda s, c: bool(re.search(r"\bamazing\b", s.text)) # hard constraint

In [ ]:
scorer = BooleanScorer(b)
g = proposal * scorer

In [ ]:
tuner = DPGTuner(a2, g,
        context="",
        n_gradient_steps=799,
        n_samples_per_step=10000,
        sampling_size=500,
        scoring_size=500,
        divergence_evaluation_interval=10)

ConsoleLogger(tuner)

tuner.tune()

In [ ]:
a2.save(path="models/amazing/wsdpg-800k")

Evaluate with above protocol

In [ ]:
token = "" # your own huggingface token
a = LMDistribution("google/gemma-2b", token=token, LLM=True)

distr = AccumulationSampler(distribution=a, total_size=500000)
samples_a, distr_a = distr.sample(sampling_size=500, context="")
print('AR =', sum([b(s, _) for s in samples_a]) / len(samples_a))

In [ ]:
a2 = LMDistribution("models/amazing/wsdpg-800k", token=token, LLM=True)

distr = AccumulationSampler(distribution=a2, total_size=200000)
samples_a2, distr_a2 = distr.sample(sampling_size=500, context="")
print('AR =', sum([b(s, _) for s in samples_a2]) / len(samples_a2))

In [ ]:
Z = sum([b(s, _) for s in samples_a]) / len(samples_a)
Z2 = sum([b(s, _) for s in samples_a2]) / len(samples_a2)
print('Z and log(Z):', Z, sp.log(Z))
print("Z' and log(Z'):", Z2, sp.log(Z2))

In [ ]:
score = []
for it, item in enumerate(samples_a):
    if b(item, _):
        score.append(a.log_score([samples_a[it]], context="") - a2.log_score([samples_a[it]], context=""))
        
estimator_g = sum(score)/len(score)

score = []
for it, item in enumerate(samples_a2[:10000]):
    if b(item, _):
        score.append(a.log_score([samples_a2[it]], context="") - a2.log_score([samples_a2[it]], context=""))

estimator_g2 = sum(score)/len(score)

In [ ]:
print("AR upgrade: ", Z2/Z)
print("KL(g|g')", estimator_g-sp.log(Z)+sp.log(Z2))
print("KL(g'|g)", -estimator_g2-sp.log(Z2)+sp.log(Z))
print("AR of a2: ", Z2)
print("KL(g|a):", -sp.log(Z))
print("KL(g'|a'):", -sp.log(Z2))
print("KL(g|a')", estimator_g-sp.log(Z))
print("KL(g'|a)", -estimator_g2-sp.log(Z2))